# PulseAD Quickstart

**PulseAD** (GradientCastPulseAD) detects anomalies by identifying significant deviations from expected behavior.

**Best for:** Detecting when metrics deviate significantly from expected patterns.

**How it works:**
1. Uses historical data as context to learn expected behavior patterns
2. Evaluates only the most recent data point(s) against those expectations
3. Flags points exceeding threshold deviations

**Important:** Historical data is required for context, but anomaly detection is performed only on the latest data point(s) in the time series you provide.

In [ ]:
import sys
sys.path.append('../..')

from gradientcast import GradientCastPulseAD
from utils.synthetic_data import generate_timestamps
import matplotlib.pyplot as plt
import numpy as np

# Replace with your API key
GRADIENTCAST_API_KEY = "your-api-key-here"

ad = GradientCastPulseAD(api_key=GRADIENTCAST_API_KEY)

---
## Basic Detection

Detect anomalies in a simple time series.

In [ ]:
# Sample data: first 7 points provide historical context,
# the last point (850000) is evaluated for anomalies
data = [
    {"timestamp": "12/01/2024, 12:00 PM", "value": 1500000},  # Historical context
    {"timestamp": "12/01/2024, 01:00 PM", "value": 1520000},  # Historical context
    {"timestamp": "12/01/2024, 02:00 PM", "value": 1510000},  # Historical context
    {"timestamp": "12/01/2024, 03:00 PM", "value": 1530000},  # Historical context
    {"timestamp": "12/01/2024, 04:00 PM", "value": 1525000},  # Historical context
    {"timestamp": "12/01/2024, 05:00 PM", "value": 1515000},  # Historical context
    {"timestamp": "12/01/2024, 06:00 PM", "value": 1540000},  # Historical context
    {"timestamp": "12/01/2024, 07:00 PM", "value": 850000},   # <-- Evaluated for anomaly (~45% drop)
]

# Detect anomalies - uses history for context, evaluates latest point(s)
result = ad.detect({"user_count": data})

print(f"Anomalies detected: {result.has_anomaly}")
print(f"Number of anomalies: {len(result.anomalies)}")

In [ ]:
# Examine the anomalies
for anomaly in result.anomalies:
    print(f"\nAnomaly at {anomaly.timestamp}:")
    print(f"  Actual value:    {anomaly.actual_value:,}")
    print(f"  Predicted value: {anomaly.predicted_value:,.0f}")
    print(f"  Delta:           {anomaly.delta:,.0f}")
    print(f"  Percent delta:   {anomaly.percent_delta}")
    print(f"  Threshold:       {anomaly.threshold}")

---
## Understanding the Response

In [ ]:
# All results (anomalies and non-anomalies)
print("All detection results:")
for r in result.results:
    status = "ANOMALY" if r.is_anomaly else "Normal"
    print(f"  {r.timestamp}: {r.actual_value:,} ({status})")

In [ ]:
# Convert to DataFrame for analysis
df = result.to_dataframe()
print("\nDataFrame columns:", list(df.columns))
df

---
## Visualizing Results

In [ ]:
# Extract values for plotting
timestamps = [d["timestamp"].split(",")[1].strip() for d in data]
values = [d["value"] for d in data]
anomaly_mask = [r.is_anomaly for r in result.results]

plt.figure(figsize=(12, 5))

# Plot the time series
plt.plot(timestamps, values, 'b-o', markersize=8, label='Actual Values')

# Highlight anomalies
anomaly_x = [t for t, m in zip(timestamps, anomaly_mask) if m]
anomaly_y = [v for v, m in zip(values, anomaly_mask) if m]
plt.scatter(anomaly_x, anomaly_y, c='red', s=200, zorder=5, 
            label='Anomaly', edgecolors='darkred', linewidths=2)

# Add expected values line
expected = [r.predicted_value for r in result.results]
plt.plot(timestamps, expected, 'g--', alpha=0.7, label='Expected')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('PulseAD: Deviation-Based Anomaly Detection')
plt.legend()
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## Multi-Dimension Detection

Monitor multiple metrics simultaneously.

In [ ]:
# Generate timestamps
timestamps = generate_timestamps(10, freq='H')

# Create multi-dimensional data
multi_data = {
    "api_requests": [
        {"timestamp": ts, "value": int(1000000 + np.random.normal(0, 50000))}
        for ts in timestamps[:-1]
    ] + [{"timestamp": timestamps[-1], "value": 400000}],  # Anomaly
    
    "active_users": [
        {"timestamp": ts, "value": int(500000 + np.random.normal(0, 25000))}
        for ts in timestamps  # No anomaly here
    ]
}

# Detect across all dimensions
result = ad.detect(multi_data)

print(f"Total anomalies: {len(result.anomalies)}")
for a in result.anomalies:
    print(f"  {a.dimension}: {a.percent_delta} deviation at {a.timestamp}")

---
## Key Concepts

### Detection Logic

An anomaly is flagged when **both** conditions are met:

1. **Percent deviation exceeds threshold** (default: 15%)
   ```
   |actual - expected| / expected > threshold
   ```

2. **Value exceeds minimum threshold** (default: 100,000)
   - Filters out noise on low-volume metrics

### When to Use PulseAD

**Good for:**
- Metrics with regular patterns
- Detecting sudden drops or spikes
- Monitoring user counts, traffic, revenue

**Consider DensityAD instead for:**
- Highly irregular patterns
- Need for severity classification
- Pattern-based anomalies

**Next:** [Tuning Thresholds](02_tuning.ipynb)